# Relatório da disciplina de Algebra Linear Computacional (CKP8122)
 - Aluno: Madson Luiz Dantas Dias
 - Professor: Creto Augusto Vidal

## Lista de implementações
### métodos iterativos
 6. Jacobi
 7. Gauss-Seidel
 8. successive over-relaxation
 9. gradientes conjugados

In [1]:
from Matrix import *

## Métodos iterativos
### Jacobi
#### Definição
Considere um sistema linear $\mathbf{A}\boldsymbol{x} = \boldsymbol{b}$, em que $\mathbf{A}\in\mathcal{R}^{n\times n}$, de modo que ela é definida como $\mathbf{A} = \mathbf{D} + \mathbf{H}$, em que $\mathbf{D} = \text{diag}\{a_{ii}\}$, $\forall i = 1,\dots, n$, e $\mathbf{H}$ é definida de tal modo que $h_{ij} = a_{ij}$, $\forall i,j=1,\dots,n$, $i\neq j$. O sistema linear apresentado então pode ser reescrito como
\begin{equation}
    (\mathbf{D} + \mathbf{H})\boldsymbol{x} = \boldsymbol{b} \iff \mathbf{D}\boldsymbol{x} = \boldsymbol{b} - \mathbf{H}\boldsymbol{x} \iff \boldsymbol{x} = \mathbf{D}^{-1}(\boldsymbol{b} - \mathbf{H}\boldsymbol{x}).
\end{equation}
Como na equação acima, $\boldsymbol{x}$ depende do próprio $\boldsymbol{x}$, podemos retirar um método iterativo que consiste em: 
\begin{equation}
    \boldsymbol{x}^{(t+1)} = \mathbf{D}^{-1}(\boldsymbol{b} - \mathbf{H}\boldsymbol{x}^{(t)}),
\end{equation}
que pode ser decomposto em
\begin{equation}
    x_i^{(t+1)} = \frac{1}{a_{ii}}\left(b_i - \sum_{j\neq i}a_{ij}x_j^{(t)}\right),\quad\forall i = 1,\dots,n.
\end{equation}


#### Código python
```Python
def jacob(self, b, K):
    M  = self.shape[1]
    x_old = zeros((1,M))
    x_new = zeros((1,M))

    for k in range(K):
        x_old = copy.deepcopy(x_new)
        for i in range(M):
            list_index = set(range(M)) - {i}
            sum_1 = self[i,list_index].dot(x_old[0,list_index]).sum(axis=1).to_number()
            x_new[0,i] = (b[i,0] - sum_1) * (1 / self[i,i])
    return x_new
```

In [7]:
A = Matrix([[4,-1,1],
            [4,-8,1],
            [-2,1,5]])
b = Matrix([[  7],
            [-21],
            [ 15]])

x = A.jacob(b, 7)

print("x = ")
print(x)

x = 

|    2.00     4.00     3.00 |



### Gauss-seidel

#### Definição
Considere um sistema linear $\mathbf{A}\boldsymbol{x} = \boldsymbol{b}$, em que $\mathbf{A}\in\mathcal{R}^{n\times n}$, de modo que ela é definida como $\mathbf{A} = \mathbf{D} + \mathbf{L} + \mathbf{U}$, em que $\mathbf{D} = \text{diag}\{a_{ii}\}$, $\forall i = 1,\dots, n$, e $\mathbf{L}$ é uma matriz triangular inferior formada pelos elementos da parte inferior da matriz \mathbf{A} e $\mathbf{U}$ é uma matriz triangular superior formada pelos elementos da parte superior da matriz \mathbf{A}. Dessa forma, assim como no método de Jacobi, o sistema linear apresentado então pode ser reescrito como
\begin{equation}
    (\mathbf{D} + \mathbf{L} + \mathbf{U})\boldsymbol{x} = \boldsymbol{b} \iff (\mathbf{D} + \mathbf{L})\boldsymbol{x} = \boldsymbol{b} - \mathbf{U}\boldsymbol{x} \iff \boldsymbol{x} = (\mathbf{D} + \mathbf{L})^{-1}(\boldsymbol{b} - \mathbf{U}\boldsymbol{x}).
\end{equation}
Como na equação acima, $\boldsymbol{x}$ depende do próprio $\boldsymbol{x}$, podemos retirar um método iterativo que consiste em:
\begin{equation}
    \boldsymbol{x}^{t+1} = (\mathbf{D} + \mathbf{L})^{-1}(\boldsymbol{b} - \mathbf{U}\boldsymbol{x}^{(t)}).
\end{equation}
que pode ser decomposta em
\begin{equation}
    x_i^{(t+1)} = \frac{1}{a_{ii}}\left(b_i - \sum_{j<i}a_{ij}x_j^{(t+1)} - \sum_{j>i}a_{ij}x_j^{(t)}\right),\quad\forall i = 1,\dots,n.
\end{equation}


#### Código Python
```Python
def gauss_seidel(self, b, K):
    M  = self.shape[1]
    x_old = zeros((1,M))
    x_new = zeros((1,M))
    for k in range(K):
        x_old = copy.deepcopy(x_new)
        for i in range(M):

            sum_1 = self[i,:i].dot(x_new[0,:i]).sum(axis=1).to_number()
            sum_2 = self[i,i+1:].dot(x_old[0,i+1:]).sum(axis=1).to_number()
            x_new[0,i] = (b[i,0] - sum_1 - sum_2) * (1 / self[i,i])
    return x_new
```

In [41]:
A = Matrix([[4,-1,1],
            [4,-8,1],
            [-2,1,5]])
b = Matrix([[  7],
            [-21],
            [ 15]])

x = A.gauss_seidel(b, 7)

print("x = ")
print(x)

x = 

|    2.00     4.00     3.00 |



### Successive Over Relaxation

#### Definição
O método Successive Over Relaxation é uma variante do método de Gauss-sedel, resultando em convergência mais rápida. Neste caso, uma constante $\omega > 0$ é adicionada aos dois lados da equação, de tal modo que 
$$
\boldsymbol{x} = (\mathbf{D} + \omega\mathbf{L})^{-1} (\omega\boldsymbol{b} - [\omega\mathbf{U} + (\omega - 1)\mathbf{D}]\boldsymbol{x}).
$$
que pode ser reescrito de como:
\begin{equation}
    \boldsymbol{x}^{(t+1)} = (\mathbf{D} + \omega\mathbf{L})^{-1}(\omega\boldsymbol{b} - [\omega\mathbf{U} + (\omega - 1)\mathbf{D}]\boldsymbol{x}^{(t)}).
\end{equation}
e decomposto em
\begin{equation}
    x_i^{(t+1)} = (1-\omega)x_i^{(t)} - \frac{\omega}{a_{ii}}\left(b_i - \sum_{j<i}a_{ij}x_j^{(t+1)} - \sum_{j>i}a_{ij}x_j^{(t)}\right),\quad\forall i = 1,\dots,n.
\end{equation}

#### Código Python
```Python
def successive_over_relaxation(self, b, K, omega):
    M  = self.shape[1]
    x_old = zeros((1,M))
    x_new = zeros((1,M))
    for k in range(K):
        x_old = copy.deepcopy(x_new)
        for i in range(M):

            sum_1 = self[i,:i].dot(x_new[0,:i]).sum(axis=1).to_number()
            sum_2 = self[i,i+1:].dot(x_old[0,i+1:]).sum(axis=1).to_number()

            x_new[0,i] = (1 - omega)*x_old[0,i] + (b[i,0] - sum_1 - sum_2) * (omega / self[i,i])
    return x_new
```

In [38]:
A = Matrix([[4,-1,1],
            [4,-8,1],
            [-2,1,5]])
b = Matrix([[  7],
            [-21],
            [ 15]])

x = A.successive_over_relaxation(b, 3, 1.3)

print("x = ")
print(x)

x = 

|    2.04     4.04     3.08 |



### Descida mais íngrime

#### Definição
Considere um sistema linear $\mathbf{A}\boldsymbol{x} = \boldsymbol{b}$, em que $\mathbf{A}\in\mathcal{R}^{n\times n}$ é uma matriz simétrica e positiva definita e $\boldsymbol{b}\in\mathcal{R}^{n}$. O método dos gradientes conjutados resolve o seguinte problema de otimização

\begin{equation}
    \boldsymbol{x}^\star =\arg \min_{\boldsymbol{x}} \frac{1}{2}\boldsymbol{x}^T\mathbf{A}\boldsymbol{x} - \boldsymbol{x}^T\boldsymbol{b} = \arg \min_{\boldsymbol{x}} \Phi.
\end{equation}

Métodos de gradiente são métodos de iterativos de otimização que utilizam informação do gradiente da função custo para atualização da solução atual, desse modo

$$
\boldsymbol{x}^{t+1} = \boldsymbol{x}^{t} - \alpha^{t} \nabla_\boldsymbol{x}\Phi
$$
em que $\alpha_t > 0$ é um passo.

Para o sistema de equações apresentado, tem-se apenas um minimizador $\boldsymbol{x}^\star$ que satisfaça
\begin{equation}
    \nabla_\boldsymbol{x}\Phi = \mathbf{A}\boldsymbol{x}^{(t)} - \boldsymbol{b} = \boldsymbol{w}^{(t)}.
\end{equation}

o passo é definido como 

$$
\alpha_t = \frac{(\boldsymbol{w}^{(t)})^T\boldsymbol{w}^{(t)}}{(\boldsymbol{w}^{(t)})^T\mathbf{A}\boldsymbol{w}^{(t)}}
$$

#### Código Python
```Python
def steepest_descent(self, b, K=1000):
    x = b
    b_hat = self * x
    w = b - b_hat
    for k in range(K):
        if w.norm() < 10**-6:
            return x
        alpha = (w.transpose() * w) / (w.transpose() * self * w)
        x = x + (w * alpha)
        b_hat = self * x
        w = b - b_hat
    return x  
```

In [5]:
A = Matrix([[10, 1, 0],
            [ 1,10, 1],
            [ 1, 1,10]])
b = Matrix([[11],
            [11],
            [ 1]])

print(A.steepest_descent(b))


|    1.00 |
|    1.01 |
|   -0.10 |

